# Getting data from OpenLab

In [2]:
import json

with open('Accounts') as json_file:  
    data = json.load(json_file)

In data: (1) Power Users; (2) Customers

In [ ]:
import requests
import csv

# It seems that most of the customers in the beginning are not loaded with transaction data 

for user in data[1][::-1][0:10]:
    # login
    url = 'https://openlab.openbankproject.com/my/logins/direct'
    headers = {'Authorization': 'DirectLogin username="'+user['user_name']+'", password="'+user['password']+'",  consumer_key="menzvlhllhpwbtxav5x1dtwxufczc1fcvlh4ss3v"',
               'Content-Type': 'application/json'}
    r=requests.post(url, headers=headers)
    tok = json.loads(r.content.decode('utf-8'))['token']
    # get accounts held by user
    baseURL = 'https://openlab.openbankproject.com'
    url = baseURL + '/obp/v3.1.0/banks/hsbc.02.hk.hsbc/accounts-held'
    headers ={ 'Authorization': 'DirectLogin token="'+ tok +'"'}
    r=requests.get(url, headers=headers)

    result=json.loads(r.content.decode('utf-8'))
    accounts = result['accounts']
    if(len(accounts)<1):
        pass
    print('processing: '+user['user_name'])
    # Get transaction for each account
    for ind in range(len(accounts)):
        account = accounts[ind]
        
        # Get account balance
        url = 'https://openlab.openbankproject.com/obp/v3.1.0/banks/hsbc.02.hk.hsbc/accounts/'+ account['id'] +'/owner/account'
        headers = {
            #'limit' : '100',
            #'to_date' : '2019-06-12T00:00:00.000Z',
            'Authorization': 'DirectLogin token="'+ tok +'"'
        }
        r=requests.get(url, headers=headers)
        results=json.loads(r.content.decode('utf-8'))
        print(results)
        # Get transaction
        url = 'https://openlab.openbankproject.com/obp/v3.1.0/banks/hsbc.02.hk.hsbc/accounts/'+ account['id'] +'/owner/transactions'
        headers = {
            'limit' : '100',
            'to_date' : '2019-06-11T00:00:00.000Z',
            'Authorization': 'DirectLogin token="'+ tok +'"'
        }
        r=requests.get(url, headers=headers)
        
        print('no account: ' + str(ind+1))
        fileName = 't_'+user['user_name']+'a_'+str(ind)+'.csv'
        results=json.loads(r.content.decode('utf-8'))
        
        fp=open(fileName, 'a+')
        fp.write('description,completed,value,new_balance\n')
        for result in results['transactions']:
            result=result['details']
            a=[result[x] for x in ['description', 'completed', 'value', 'new_balance']]
            a[2]=a[2]['amount'] + a[2]['currency']
            a[3]=a[3]['amount'] + a[3]['currency']
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(a)
        fp.close()


processing: Eleanor.Hk.02
{'id': '8e23029c-9ff7-4938-8e9c-61bffcc89ce4', 'bank_id': 'hsbc.02.hk.hsbc', 'label': 'Eleanor.Hk.02 M35 17..841', 'number': '17703365841', 'owners': [{'id': 'Eleanor.Hk.02', 'provider': 'https://openlab.openbankproject.com', 'display_name': 'Eleanor.Hk.02'}], 'type': 'BUSINESS CURRENT', 'balance': {'currency': 'HKD', 'amount': '80532.96'}, 'account_routings': [{'scheme': 'OBP', 'address': '8e23029c-9ff7-4938-8e9c-61bffcc89ce4'}], 'account_rules': []}
no account: 1
processing: Elise.Hk.02
{'id': '943847d4-13e3-4667-9850-48d816049215', 'bank_id': 'hsbc.02.hk.hsbc', 'label': 'Elise.Hk.02 M35 17..812', 'number': '17203174812', 'owners': [{'id': 'Elise.Hk.02', 'provider': 'https://openlab.openbankproject.com', 'display_name': 'Elise.Hk.02'}], 'type': 'CURRENT PLUS', 'balance': {'currency': 'HKD', 'amount': '5016.82'}, 'account_routings': [{'scheme': 'OBP', 'address': '943847d4-13e3-4667-9850-48d816049215'}], 'account_rules': []}
no account: 1
processing: Jean-paul

In [119]:
# Each transaction consist of:
results['transactions'][0].keys()

# Only used details here

dict_keys(['id', 'this_account', 'other_account', 'details', 'metadata'])

In [ ]:
results['transactions'][0]['details']